# Build a RAG System with Grounding using LangChain and Agent Builder Data Stores

In [ ]:
! pip install -q --user google-cloud-aiplatform google-cloud-discoveryengine langchain-google-vertexai langchain-google-community

In [ ]:
# Restart kernel after packages are installed so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import vertexai
import langchain

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)


print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {vertexai.__version__}")

# Prerequisites

You need to create a Data Store and Search app in Agent Builder.

The Data Store should use the following Cloud Storage location:

`gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs`

Make sure to enable Enterprise features when creating the Search App.

In [ ]:
# Find your Data Store ID in the Agent Builder console.
DATA_STORE_ID = "qna-unstructured-datastore_1717079477615"  # @param {type:"string"}
DATA_STORE_LOCATION = "global"  # @param {type:"string"}

MODEL = "gemini-1.0-pro"  # @param {type:"string"}

if PROJECT_ID == "YOUR_PROJECT_ID" or DATA_STORE_ID == "YOUR_DATA_STORE_ID":
    raise ValueError(
        "Please set the PROJECT_ID, DATA_STORE_ID constants to reflect your environment."
    )

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from langchain_google_vertexai import VertexAI
from langchain_google_community import VertexAISearchRetriever
from langchain_google_community import VertexAIMultiTurnSearchRetriever

In [ ]:
llm = VertexAI(model_name=MODEL)

# The retriever has the magic to search the
# Data Store based on the user's question
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DATA_STORE_LOCATION,
    data_store_id=DATA_STORE_ID,
    get_extractive_answers=True,
    max_documents=10,
    max_extractive_segment_count=1,
    max_extractive_answer_count=5,
)

# Use RetrievalQA to Ask the Question

In [ ]:
search_query = "What was Alphabet's Revenue in Q2 2021?"

retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)
retrieval_qa.invoke(search_query)

## Include Grounding Data

Add `return_source_documents=True` to see the documents used to answer the question

In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

results = retrieval_qa.invoke(search_query)

print("*" * 80)
print(results["result"])
print("*" * 80)
for doc in results["source_documents"]:
    print("-" * 80)
    print(doc.page_content)

## Include Grounding Sources with the Results

In [ ]:
retrieval_qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)

retrieval_qa_with_sources.invoke(search_query, return_only_outputs=True)

## Multi-Turn Retriever allows for followup questions

In [ ]:
multi_turn_retriever = VertexAIMultiTurnSearchRetriever(
    project_id=PROJECT_ID, location_id=DATA_STORE_LOCATION, data_store_id=DATA_STORE_ID
)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversational_retrieval = ConversationalRetrievalChain.from_llm(
    llm=llm, retriever=multi_turn_retriever, memory=memory
)

search_query = "What were alphabet revenues in 2022?"

result = conversational_retrieval.invoke(search_query)
print(result["answer"])

In [ ]:
new_query = "What about costs and expenses?"
result = conversational_retrieval.invoke(new_query)
print(result["answer"])

In [ ]:
new_query = "Is this more than in 2021?"

result = conversational_retrieval.invoke(new_query)
print(result["answer"])

## Creating a Custom Prompt

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

# This just shows the default prompt
print(qa.combine_documents_chain.llm_chain.prompt.template)

In [ ]:
# Create a custom prompt template.
# Note: it instructs the model to return a 1 word answer
prompt_template = """Use the context to answer the question at the end.
You must always use the context and context only to answer the question. Never try to make up an answer. If the context is empty or you do not know the answer, just say "I don't know".
The answer should consist of only 1 word and not a sentence.

Context: {context}

Question: {question}
Helpful Answer:
"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Specify the custom prompt template in the chain
qa_chain = RetrievalQA.from_llm(
    llm=llm, prompt=prompt, retriever=retriever, return_source_documents=True
)

In [ ]:
# Show the custom prompt template just to show it worked.
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

In [ ]:
search_query = "Were 2020 EMEA revenues higher than 2020 APAC revenues?"

results = qa_chain.invoke(search_query)

print("*" * 80)
print(results["result"])
print("*" * 80)
for doc in results["source_documents"]:
    print("-" * 80)
    print(doc.page_content)